## 🧮 Mathematical Foundation

### Architecture

**Encoder**: $z = f_{enc}(x; \theta_{enc})$ maps input $x \in \mathbb{R}^d$ to latent $z \in \mathbb{R}^k$ (k << d)

**Decoder**: $\hat{x} = f_{dec}(z; \theta_{dec})$ reconstructs from latent space

**Training Loss** (MSE):
$$L = \frac{1}{n}\sum_{i=1}^{n} ||x_i - \hat{x}_i||^2$$

**Anomaly Score**:
$$s(x) = ||x - f_{dec}(f_{enc}(x))||^2$$

Normal points: low reconstruction error  
Anomalies: high reconstruction error (didn't learn these patterns)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

sns.set_style('whitegrid')
np.random.seed(42)
torch.manual_seed(42)

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim=8):
        super(AutoEncoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, latent_dim)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim)
        )
    
    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

print("✅ AutoEncoder architecture defined")

In [ ]:
# Generate data
X_normal, _ = make_blobs(n_samples=1000, centers=1, n_features=10, random_state=42)
X_anomalies = np.random.uniform(low=-8, high=8, size=(100, 10))

# Standardize
scaler = StandardScaler()
X_normal_scaled = scaler.fit_transform(X_normal)
X_anomalies_scaled = scaler.transform(X_anomalies)

# Train autoencoder on normal data only
X_train_tensor = torch.FloatTensor(X_normal_scaled)
model = AutoEncoder(input_dim=10, latent_dim=3)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
losses = []
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    recon = model(X_train_tensor)
    loss = criterion(recon, X_train_tensor)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

print(f"✅ Training complete. Final loss: {losses[-1]:.4f}")

In [ ]:
# Compute reconstruction errors
model.eval()
with torch.no_grad():
    X_test = np.vstack([X_normal_scaled, X_anomalies_scaled])
    X_test_tensor = torch.FloatTensor(X_test)
    recon_test = model(X_test_tensor).numpy()
    errors = np.mean((X_test - recon_test)**2, axis=1)

y_true = np.array([1]*len(X_normal_scaled) + [-1]*len(X_anomalies_scaled))

# Set threshold (95th percentile of normal errors)
threshold = np.percentile(errors[:len(X_normal_scaled)], 95)
y_pred = np.where(errors > threshold, -1, 1)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.hist(errors[:len(X_normal_scaled)], bins=30, alpha=0.6, label='Normal', color='blue')
ax.hist(errors[len(X_normal_scaled):], bins=30, alpha=0.6, label='Anomalies', color='red')
ax.axvline(threshold, color='green', linestyle='--', linewidth=2, label=f'Threshold={threshold:.3f}')
ax.set_xlabel('Reconstruction Error')
ax.set_ylabel('Frequency')
ax.set_title('AutoEncoder: Reconstruction Error Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
y_true_binary = (y_true == -1).astype(int)
fpr, tpr, _ = roc_curve(y_true_binary, errors)
auc = roc_auc_score(y_true_binary, errors)
ax.plot(fpr, tpr, linewidth=2, label=f'AUC={auc:.3f}')
ax.plot([0, 1], [0, 1], 'k--', linewidth=1)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Performance:")
print(classification_report(y_true, y_pred, target_names=['Normal', 'Anomaly']))
print(f"\nAUC: {auc:.3f}")

## 🏭 Semiconductor Application

### 📝 High-Dimensional Parametric Test Anomaly Detection

In [ ]:
# Generate realistic semiconductor test data (20 parameters)
np.random.seed(42)
n_params = 20
n_normal = 2000
n_anomaly = 100

# Normal devices: correlated parameters
mean_normal = np.random.uniform(1.5, 2.5, n_params)
cov_normal = np.eye(n_params) * 0.05
X_psv_normal = np.random.multivariate_normal(mean_normal, cov_normal, n_normal)

# Anomalous devices: parameter drift
X_psv_anomaly = X_psv_normal[:n_anomaly].copy()
X_psv_anomaly[:, :5] += np.random.uniform(0.5, 1.5, (n_anomaly, 5))  # Drift in first 5 params

# Scale
scaler_psv = StandardScaler()
X_psv_normal_scaled = scaler_psv.fit_transform(X_psv_normal)
X_psv_anomaly_scaled = scaler_psv.transform(X_psv_anomaly)

# Train autoencoder
X_train_psv = torch.FloatTensor(X_psv_normal_scaled)
model_psv = AutoEncoder(input_dim=n_params, latent_dim=5)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_psv.parameters(), lr=0.001)

for epoch in range(100):
    model_psv.train()
    optimizer.zero_grad()
    recon = model_psv(X_train_psv)
    loss = criterion(recon, X_train_psv)
    loss.backward()
    optimizer.step()

# Test
model_psv.eval()
with torch.no_grad():
    X_test_psv = np.vstack([X_psv_normal_scaled[:500], X_psv_anomaly_scaled])
    X_test_psv_tensor = torch.FloatTensor(X_test_psv)
    recon_psv = model_psv(X_test_psv_tensor).numpy()
    errors_psv = np.mean((X_test_psv - recon_psv)**2, axis=1)

y_true_psv = np.array([1]*500 + [-1]*n_anomaly)
threshold_psv = np.percentile(errors_psv[:500], 95)
y_pred_psv = np.where(errors_psv > threshold_psv, -1, 1)

# Visualize
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(errors_psv[:500], bins=30, alpha=0.6, label='Normal Devices', color='blue')
plt.hist(errors_psv[500:], bins=30, alpha=0.6, label='Anomalous Devices', color='red')
plt.axvline(threshold_psv, color='green', linestyle='--', linewidth=2, label=f'Threshold')
plt.xlabel('Reconstruction Error')
plt.ylabel('Number of Devices')
plt.title('Parametric Test Anomaly Detection (20 Parameters)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Feature importance (average reconstruction error per feature)
feature_errors = np.mean((X_test_psv[500:] - recon_psv[500:])**2, axis=0)
plt.bar(range(n_params), feature_errors, color='coral')
plt.xlabel('Parameter Index')
plt.ylabel('Avg Reconstruction Error')
plt.title('Parameter-wise Anomaly Contribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n⚠️ Anomalous Devices Detected:")
print(f"   Total: {(y_pred_psv == -1).sum()} / {len(X_test_psv)}")
print("\n📊 Performance:")
print(classification_report(y_true_psv, y_pred_psv, target_names=['Normal', 'Anomaly']))

## 🎯 Project Ideas

### Post-Silicon Projects

1. **Wafer Map Anomaly Detector** 💰 $8M+ Yield Improvement
   - Train on normal wafer spatial patterns, detect systematic defects
   - Features: 2D die coordinates + parametric test values
   - Business: Early fab process issue detection

2. **Multi-Site Test Correlation Monitor** 💰 $12M+ Quality
   - 50+ parametric tests, detect novel failure modes
   - AutoEncoder learns normal parameter correlations
   - Business: Improve test coverage, reduce escapes

3. **Time-Series Waveform Anomalies** 💰 $5M+ Debug Time
   - LSTM AutoEncoder on test waveforms
   - Detect subtle signal integrity issues
   - Business: Faster failure analysis

4. **Cross-Product Defect Discovery** 💰 $15M+ Portfolio
   - Train per-product autoencoders
   - Transfer learning for new products
   - Business: Accelerate new product ramp

### General Projects

5. **Network Intrusion Detection** 💰 $30M+ Security
6. **Medical Image Anomalies** 💰 $100M+ Healthcare
7. **Industrial Sensor Monitoring** 💰 $20M+ Downtime
8. **Financial Transaction Fraud** 💰 $150M+ Fraud Prevention

## 🔍 Key Takeaways

### ✅ When to Use AutoEncoders
- **High-dimensional data** (>50 features): Learns compressed representations
- **Complex patterns**: Captures non-linear correlations via deep networks
- **Unlabeled data**: Unsupervised, trains on normal only
- **Feature learning**: Automatic feature extraction (no manual engineering)

### ❌ Limitations
- **Training time**: Requires GPU for large datasets
- **Hyperparameters**: Architecture, latent dim, learning rate tuning needed
- **Overfitting risk**: May memorize training data (use regularization)
- **Black box**: Less interpretable than tree-based methods

### 🔧 Best Practices
1. **Always standardize** inputs (zero mean, unit variance)
2. **Latent dimension**: Start with d/4 to d/2 (compression ratio 2-4x)
3. **Threshold**: 95th-99th percentile of training errors
4. **Validation**: Use contaminated validation set to tune threshold
5. **Regularization**: Dropout, L2 weight decay to prevent overfitting

### 📊 Comparison

| Method | Speed | High-D | Interpretability | Best For |
|--------|-------|--------|------------------|----------|
| **AutoEncoder** | Slow train, fast inference | ✅ Excellent | ❌ Low | Complex patterns, images |
| **Isolation Forest** | ✅ Fast | ✅ Good | ⚠️ Medium | Large data, speed |
| **One-Class SVM** | ❌ Slow | ⚠️ Medium | ✅ Good | Novelty, small data |

### 🚀 Next Steps
- Variational AutoEncoders (VAE) for probabilistic anomaly scores
- LSTM AutoEncoders for time-series anomalies
- Convolutional AutoEncoders for image/wafer map anomalies